# Homework exercise 1
## Deadline: upload to Moodle by 19 November 18:00 h

__Suggestion: take this notebook and simply add your code and explanations. Your submission needs to include your code's output (even if the code throws an error).__

If you prefer to use .py files, you are expected to also include a PDF containing the output of your code and your explanations. Still, the code needs to be in a form that can be easily run on another computer.

If you use any file paths, assign paths to variables at the beginning of your code and use those variables when later referring to the paths.

__Name :__ Moritz-Jakob Leithner


The name of the file that you upload should be named *Homework1_YourLastName_YourStudentID*.

Reminder: you are required to attend class on 20 November to earn points for this homework exercise unless you have a valid reason for your absence.

You are encouraged to work on this exercise in teams of up to three students. If any part of the questions is unclear, please ask on the Moodle forum.

#### Simulating a stock market
Consider a financial market consisting of one or more stock exchanges. In this market, only one asset is traded (to keep it simple). 

Each stock exchange operates as a limit order book. I.e., if an investor Alice submits, e.g., a buy order, she specifies the maximum price she is willing to pay (called the limit price) and the number of shares she would like to trade (we will set the number of shares to 1 for simplicity). If there already is an existing sell order in the order book (previously submitted by another trader we will call Bob) with a limit price below or equal to Alice's limit price, a trade happens at Bob's limit price. If there is no such existing order, Alice's order enters the limit order book to wait for a seller who might be willing to trade at her limit price. Assume, for simplicity and contrary to real markets, that an order cannot be cancelled or modified after it has been submitted.

This problem set consists of tasks asking you to
* implement the functionality of a stock exchange operating as a limit order market
* generate random data representing orders submitted to the market
* simulate a market consisting of one or more stock exchanges using the randomly generated order flow
* finally interpret the output generated by your code

__You are expected to implement your solution using basic Python functionality plus numpy (in particular, for the generation of random numbers).__

1. Implementing a limit oder market

You are strongly encouraged to implement the limit order market as a class, so that you can later easily generate exchanges with slightly varying parameters.

The limit order market needs to offer the following functionality:

* access to the best bid and the best ask price (as attributes of the market or via methods)
* processing of incoming orders, where orders have the parameters `limit_price`, `side` (i.e. buy or sell), `size` (i.e. number of shares), and an `order_id` (i.e. an integer identifying an order). Processing constitutes of 
    * updating the order book (i.e. inserting the new order if it does not trade immediately, otherwise remove or modify the order against which the new order trades)
    * returning information on whether a trade happened and, if so, return trade price, size, and the IDs of the order that traded against one another
    
The only parameter pertaining to the order book is the tick size, which is a measure of the granularity of the prices the exchange allows. E.g., a tick size of 0.01 means that limit prices must be numbers expressed in whole cents.     

__Note:__ if there are multiple orders that would match with the new order, the buy order with the highest limit price/sell order with the lowest limit price will trade against the new order. If multiple orders at the same price exist in the order book, the order submitted the earliest get to trade first. This rule is called price-time priority and is the most common, though not the only, rule applied in real limit order books.

In [1]:
class OrderBook(object):
    def __init__(self):
        self.asks = []
        self.bids = []
        self.max_bid =  None
        self.min_ask =  None

    def add_order(self, order):
        if order.side == 'sell':
            if self.max_bid is None or order.limit_price > self.max_bid:
                if self.min_ask is None or order.limit_price < self.min_ask:
                    self.min_ask = order.limit_price
                    self.asks.append(order)
                else:  
                    self.asks.append(order)
            else:
                self.match_order(order, self.bids)
  
        elif order.side == 'buy':
            if self.min_ask is None or order.limit_price < self.min_ask:
                if self.max_bid is None or order.limit_price > self.max_bid:
                    self.max_bid = order.limit_price
                    self.bids.append(order)
                else:
                    self.bids.append(order)
            else:
               self.match_order(order, self.asks)


    def match_order(self, new_order, orders):
        for order in orders:
            if new_order.side == 'buy' and order.limit_price <= new_order.limit_price:
                trade_price = min(self.min_ask, new_order.limit_price)
                trade_size = new_order.size
                trade = Trade(new_order.ID, order.ID, trade_price, trade_size)
                print("Trade: %s" % trade)
                orders.remove(order)
                del new_order
                break
            elif new_order.side == 'sell' and order.limit_price >= new_order.limit_price:
                trade_price = min(self.min_ask, new_order.limit_price)
                trade_size = new_order.size
                trade = Trade(new_order.ID, order.ID, trade_price, trade_size)
                print("Trade: %s" % trade)
                orders.remove(order)
                del new_order
                break
        # Update max_bid and min_ask
        self.max_bid = max([bid.limit_price for bid in self.bids]) if self.bids else None
        self.min_ask = min([ask.limit_price for ask in self.asks]) if self.asks else None
        

        
    def display(self):
        print("Total number of orders: %s" % Order.number)
        print("Max bid: %s" % self.max_bid)
        print("Min ask: %s" % self.min_ask)
        print("Bids: ")
        for bid in self.bids:
            print(bid)
        print("Asks: ")
        for ask in self.asks:
            print(ask)
        
order_book = OrderBook()

class Trade(object):
    def __init__(self, buy_order_id, sell_order_id, price, size):
        self.buy_order_id = buy_order_id
        self.sell_order_id = sell_order_id
        self.price = price
        self.size = size

    def __str__(self):
        return "Buy Order ID: %s, Sell Order ID: %s, Price: %s, Size: %s" % (self.buy_order_id, self.sell_order_id, self.price, self.size)
    

class Order(object):
    number = 0

    def __init__(self,limit_price):
        self.limit_price = limit_price
        self.ID = Order.number + 1
        Order.number += 1
        print('Order (ID: %s) created' %(self.ID))
    
    def __str__(self):
        return " Limit Price: %s, Side: %s, Order ID: %s" %(self.limit_price, self.side, self.ID)

    def __del__(self):
        print('Order (ID: %s) deleted' %(self.ID))


class Ask(Order):

        def __init__(self, limit_price, size = 1, side = 'sell'):
            Order.__init__(self, limit_price)
            self.limit_price = limit_price
            self.size = size
            self.side = side
            order_book.add_order(self)

class Bid(Order):

        def __init__(self, limit_price, size = 1, side = 'buy'):
            Order.__init__(self, limit_price)
            self.limit_price = limit_price
            self.size = size
            self.side = side
            order_book.add_order(self)



######## Mickey Mouse Example ########
order_book.display()
order1 = Ask(100.5)
order_book.display()
order2 = Ask(103)
order_book.display()
order3 = Bid(102.5)
order_book.display()
order4 = Bid(101)
order_book.display()
order5 = Ask(34)
order_book.display()
order6 = Bid(104)
order_book.display()
order7 = Ask(100)
order_book.display()
order8 = Bid(105)
order_book.display()
order9 = Bid(25)
order_book.display()

2. Generating random numbers and simulating order flow

You are asked to allow the orders submitted to the market to depend on the following parameters:

* the fair value of the security
* a private value an individual trader experiences from holding the asset. This private value reflect how much the trader likes or dislikes owning the asset

We are going to assume the following simple rule determining side and limit price: if the private value is positive (negative), the trader will submit a buy (sell) order at a price equal to the sum of fair value and private value minus (plus) a constant `k`, which we will interpret as the required "profit", rounded down (up) to the nearest tick. This rule is not intended to reflect optimal behavior by the traders but rather serves to get a relatively reasonable simulation of the trading process. `the required profit is constant for all traders? what exactly does it do? -> no one trades at their actual valuation, but at the valuation plus profit..... no trade w/o profit? or does that have smt. to do w/ the min/max ask/bids that are entered into the system?`

Write a function that generates the orders based on the parameters described above. Generate 100,000 random values following a standard normal distribution `just generate a sequence of random vars and then use it in a calculation to set up the whole thing` . Also

* generate 100,000 random numbers following the uniform distribution between 0 and 1, which we will use to determine changes to the fair value in the next step
* generate 100,000 binary random numbers with equal probability, which we will use to determine whether any change to the fair value is positive or negative
* generate 100,000 binary random numbers with equal probability, which we will use to determine the exchange to which an order is submitted if the trader is indifferent in the setting with two exchanges

3. Simulating the market

Based on what you implemented in the previous steps, please simulate a market for all combinations of the following parameters:

* Initital fair value: 100
* Constant size for all orders: 1
* Probability of an increase/decrease of the fair value by 1 in each period (1 order = 1 period): 0, 0.001, 0.01, 0.1
* Parameter `k` (see above): 0.1, 1
* Standard deviation of private values: 0.1, 1
* Exchanges: 
    * one exchange with tick size 0.01, 0.1, 1
    * two exchanges each with tick size 0.01, 0.1, 1
    * one exchange with tick size 0.01, another with tick size 0.1 
    * one exchange with tick size 0.1, another with tick size 1 

Rules for order submission with multipe exchanges:

* If the order can immediately trade on one of the two exchanges, send the order to that exchange
* If the order can immediately trade on neither of the exchanges, send the order with equal probability to either of the exchanges
* If the order can immediately trade on both exchanges, send the order to the exchange offering the better price

For each simulation, compute the total trading volume, the average bid-ask spread (simple difference between ask and bid) on each exchange, and the standard deviation of changes in trade prices. For simulations containing two exchanges, also compute the trading volumes on each exchange.

4. Try to interpret the results computed above. How do trading volume, price volatility, and (for the setting with two exchanges) the proportion of trades on each exchange depend on the parameters of the simulation?